In [ ]:
# download the weaviate client
%pip install -U weaviate-client

In [ ]:
import weaviate, os
from weaviate.config import AdditionalConfig, Timeout, ConnectionConfig
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Retrieve environment variables
CLUSTER_URL = os.getenv("CLUSTER_URL")
API_KEY = os.getenv("API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
COHERE_API_KEY = os.getenv("COHERE_API_KEY")

# Connect to Weaviate
client = weaviate.connect_to_weaviate_cloud(
	cluster_url=CLUSTER_URL,
	auth_credentials=weaviate.auth.AuthApiKey(API_KEY),
	headers={
		"X-OpenAI-Api-Key": OPENAI_API_KEY,
		"X-Cohere-Api-Key": COHERE_API_KEY
	},
	additional_config=AdditionalConfig(
		timeout=Timeout(init=30, query=60, insert=120)
			)
)

ready = client.is_ready()
server_version = client.get_meta()["version"]
client_version = weaviate.__version__

live = client.is_live()
connected = client.is_connected()

print(f"Weaviate client is ready: {ready}")
print(f"Weaviate Client Version: {client_version}")
print(f"Weaviate Server Version: {server_version}")
print(f"live: {live}")
print(f"Connected: {connected}")

In [ ]:
from weaviate.classes.config import ConsistencyLevel

collection = client.collections.get("<COL_NAME>").with_consistency_level(consistency_level=ConsistencyLevel.ALL)
total_objects = 0
for item in collection.iterator(cache_size=200):
    total_objects += 1
    print(f"Object: {total_objects}")
    print(item.uuid, item.properties)
print(f"Total objects fetched: {total_objects}")


In [ ]:
# Get the meta information of the Weaviate instance
import json
result = client.get_meta()
print(json.dumps(result, indent=2))

In [ ]:
# List the collection and its count
try:
    collections = client.collections.list_all()
    if collections:
        print("Collections in Weaviate:")
        print(f"Total collections: {len(collections)}")
    else:
        print("No collections found.")
except Exception as e:
    print(f"Error retrieving collections: {e}")

In [ ]:
# Retrieve the collection objects including the vectors
collection = client.collections.get("<COL_NAME>")
for item in collection.iterator(
    include_vector=True
):
    print(item.properties)
    print(item.vector)

In [ ]:
# Retrieve the collections & objects including the vectors in a table format
import pandas as pd

def get_all_collections_data(client):
    all_data = []
    collections = client.collections.list_all()
    
    for col_name in collections:
        try:
            collection = client.collections.get(col_name)
            collection_data = []
            
            for item in collection.iterator(include_vector=True):
                row = item.properties.copy()
                row['vector'] = item.vector
                collection_data.append(row)
            
            if collection_data:
                df = pd.DataFrame(collection_data)
                df['collection'] = col_name
                all_data.append(df)
                print(f"Successfully processed: {col_name}")
                
        except Exception as e:
            print(f"Error in {col_name}: {str(e)}")
            continue
    
    return pd.concat(all_data, ignore_index=True) if all_data else pd.DataFrame()

get_all_collections_data(client)

In [ ]:
# Check if multi-tenancy is enabled for the collections
def check_collections_multi_tenancy(client):
    try:
        collections = client.collections.list_all()
        collection_count = 0
        multi_tenancy_collections = 0

        if collections:
            collection_count = len(collections)
            print(f"Total collections: {collection_count}")
            print("-" * 50)

            for collection_name in collections:
                collection = client.collections.get(collection_name)
                print(f"Collection: {collection_name}")
                
                try:
                    # Attempt to get tenants for the collection (check if multi-tenancy is enabled)
                    tenants = collection.tenants.get()
                    
                    # If we get here without an exception, multi-tenancy is enabled
                    multi_tenancy_collections += 1
                    tenant_count = len(tenants)
                    
                    print(f"  Multi-tenancy: Enabled")
                    print(f"  Number of tenants: {tenant_count}")
                    
                    if tenant_count > 0:
                        print(f"  Tenant names:")
                        for tenant_name in tenants.keys():
                            print(f"    - {tenant_name}")
                    else:
                        print(f"  No tenants found for this collection")
                        
                except Exception as e:
                    if "multi-tenancy is not enabled" in str(e):
                        print(f"  Multi-tenancy: Disabled")
                    else:
                        print(f"  Error checking multi-tenancy: {str(e)}")
                
                print("-" * 50)
            
            print(f"Summary:")
            print(f"  Total collections: {collection_count}")
            print(f"  Collections with multi-tenancy enabled: {multi_tenancy_collections}")
            
            return {
                "collection_count": collection_count,
                "multi_tenancy_collections": multi_tenancy_collections
            }
        
        else:
            print("No collections found")
            return {
                "collection_count": 0,
                "multi_tenancy_collections": 0
            }

    except Exception as e:
        print(f"Error: {str(e)}")
        return {"error": str(e)}
    
# Run the function
check_collections_multi_tenancy(client)